In [1]:
import torch as th
import numpy as np
from tqdm.notebook import trange

In [2]:
device = "mps"
dtype = th.float32

In [3]:
x = th.linspace(
    -np.pi,
    np.pi,
    2000,
    device=device,
    dtype=dtype
)
y = th.sin(x)

In [4]:
class Polynomial3(th.nn.Module):
    def __init__(self):
        super(Polynomial3, self).__init__()
        self.a = th.nn.Parameter(th.randn(()))
        self.b = th.nn.Parameter(th.randn(()))
        self.c = th.nn.Parameter(th.randn(()))
        self.d = th.nn.Parameter(th.randn(()))

    def forward(self, x):
        return self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3

In [5]:
model = Polynomial3().to(device=device, dtype=dtype)

# Manual optimization
# Use simple gradient descent to fit the model to our data
steps = 2000
eps = 1e-6
lr = 1e-6
logging_steps = 100
for t in trange(steps):
    y_pred = model(x)
    loss = th.square(y_pred - y).sum()
    model.zero_grad()
    if t % logging_steps == logging_steps - 1:
        print(t, loss.item())
    loss.backward()
    with th.no_grad():
        for param in model.parameters():
            param -= lr * param.grad

  0%|          | 0/2000 [00:00<?, ?it/s]

99 2129.010009765625
199 1485.078369140625
299 1037.5623779296875
399 726.2747802734375
499 509.560791015625
599 358.5625
699 253.2691650390625
799 179.7897491455078
899 128.47438049316406
999 92.61219787597656
1099 67.53253936767578
1199 49.982173919677734
1299 37.69303512573242
1399 29.082855224609375
1499 23.04688835144043
1599 18.813251495361328
1699 15.842259407043457
1799 13.756295204162598
1899 12.291068077087402
1999 11.26141357421875


In [6]:
import pandas as pd
import plotly.express as px

In [7]:
x = x
y = y
y_pred = model(x)

data = pd.DataFrame({
    "x": x.cpu().numpy(),
    "y": y.cpu().numpy(),
    "y_pred": y_pred.detach().cpu().numpy()
})

In [8]:
fig = px.line(data, x="x", y=["y", "y_pred"], title="Fitting a polynomial to data")
fig.show()